# [Your name]

# Homework 3

The maximum score of this homework is 100+20 points. Grading is listed in this table:

| Grade | Score range |
| --- | --- |
| 5 | 85+ |
| 4 | 70-84 |
| 3 | 55-69 |
| 2 | 40-54 |
| 1 | 0-39 |

Most exercises include tests which should pass if your solution is correct.
However successful test do not guarantee that your solution is correct.
The homework is partially autograded using many hidden tests.
Test cells cannot be modified and empty cells cannot be deleted.

Your solution should replace placeholder lines such as:

    ### YOUR CODE HERE
    raise NotImplementedError()

You don't need to write separate functions except when the function header is predefined.
Variable names must be derived from the public test.
Please do not add new cells, they will be ignored by the autograder.

**VERY IMPORTANT** Before submitting your solution (pushing to the git repo),
run your notebook with `Kernel -> Restart & Run All` and make sure it
runs without exceptions.

If your code fails the public tests (the ones you see), you will automatically receive 0 points for that exercise. Furthermore, later tasks within the same exercise might require the output of earlier ones; failing to provide implementation for the earlier tasks in this scenario will **prevent you from earning any points** for the later ones.

## Submission

GitHub Classroom will accept your last pushed version before the deadline.
You do not need to send the homework to the instructor.

## Plagiarism

When preparing their homework, students are reminded to pay special attention to Title 32, Sections 92-93 of Code of Studies (quoted below). Any content from external sources must be stated in the students own words AND accompanied by citations. Copying and pasting from an external source should be avoided and any text copied must be placed between quotation marks. Reports that violate these rules cannot receive a passing grade.

"**Section 92**

(1) The works of another person will be used as follows: a) if a work of another person is used in whole or in part (e.g. by copying, citation, translation from another language or presentation), the source and the name of the author will be indicated if this name is included in the source or – in case of orally presented works – may be clearly identified; b) the work of another person or any part of that will be used – up to a quantity reasonably corresponding to the nature and purpose of the student work – identified as quotations.

(2) Instructors are entitled to review compliance with requirements in this article with computer programmes and databases.

(3) The use of works of another person and the acknowledgement of use will be governed by applicable laws and the relevant rules of the specific discipline.

**Section 93**

(1) If a student fails to meet rules regarding use of works of another person in whole or in part, the student work will be considered as not assessable and the student will not be allowed to obtain the credit of the concerned subject in the specific term.

(2) It will be deemed a disciplinary offence if a student – in breach of the rules regarding use of works of another person – submits or presents a work of another person fully or in a significant part verbatim (word for word) or in terms of its basic concepts or the combined version of several works of another person(s) as their own work.

(3) Based on subsection (1) of Section 52/A. of the Higher Education Act, compliance with the rules regarding the use of works of another person in a master thesis may be reviewed up to five years following the issue of the degree certificate. In case of violation of the above rules, section 52/A of the Higher Education Act will apply."

(BME Code of Studies, p.50)

## Boilerplate

In [ ]:
from functools import partial
import os
import re
import subprocess
import tempfile
import types
import urllib.request

import nltk
from nltk import Nonterminal
from nltk.tree import Tree
import numpy

def execute_commands(*cmds, fancy=True):
    """
    Starts foma end executes the specified commands.
    Might not work if there are too many...
    """
    if fancy:
        print('Executing commands...\n=====================\n')
    args = ' '.join('-e "{}"'.format(cmd) for cmd in cmds)
    output = subprocess.check_output('foma {} -s'.format(args),
                                     stderr=subprocess.STDOUT,
                                     shell=True).decode('utf-8')
    print(output)
    if fancy:
        print('=====================\n')
    
def compile_lexc(lexc_string, fst_file):
    """
    Compiles a string describing a lexc lexicon with foma. The FST
    is written to fst_file.
    """
    with tempfile.NamedTemporaryFile(mode='wt', encoding='utf-8', delete=False) as outf:
            outf.write(lexc_string)
    try:
        execute_commands('read lexc {}'.format(outf.name),
                         'save stack {}'.format(fst_file), fancy=False)
        #!foma -e "read lexc {outf.name}" -e "save stack {fst_file}" -s
    finally:
        os.remove(outf.name)
        
def apply(fst_file, words, up=True):
    """
    Applies the FST in fst_file on the supplied words. The default direction
    is up.
    """
    if isinstance(words, list):
        words = '\n'.join(map(str, words))
    elif not isinstance(words, str):
        raise ValueError('words must be a str or list')
    invert = '-i' if not up else ''
    result = subprocess.check_output('flookup {} {}'.format(invert, fst_file),
                                     stderr=subprocess.STDOUT, shell=True,
                                     input=words.encode('utf-8'))
    words = result.decode('utf-8').strip().split('\n\n')  # Skip last newline
    return [word.split('\t') for word in words]
       
apply_up = partial(apply, up=True)
apply_down = partial(apply, up=False)

def draw_all_trees(parser, sentence):
    for tree in parser.parse(sentence):
        display(tree)

Although not necessary, feel free to copy any additional boilerplate code you need from labs [9](../../course_material/09_Morphology_lab/09_Morphology_lab.ipynb#Morphology) and [10](../../course_material/10_Syntax/10_Syntax_lab.ipynb#Boilerplate) below.

# Exercise 1: Morphology (33 points)

## 1.1 Esperanto nouns (13 points)

In this exercise, you are going to write a simple foma grammar for Esperanto nouns. Esperanto, like Hungarian, is agglutinative; however, as you will see, the similarities stop there. Your grammar will not cover all the ins and outs of Esperanto noun inflection, but it will give you a "feel".

What follows are the rules of Esperanto noun declension. Your task is to write a `lexc` grammar based on them.

1. All words begin with a root, such as the animal names below (`best` just means _animal_)
   ```
   hund
   kat
   bird
   elefant
   tigr
   best
   leon
   ```
1. The nominative is formed by the _o_ suffix, e.g. _hundo_.
1. Three suffixes can be placed between the root and the _o_:
    - the feminine marker _in_: e.g. _hundino_ (bitch)
    - the diminutive marker _et_: e.g. _hundeto_ (puppy)
    - the augmentative marker _eg_: e.g. hundego (big dog)
1. There is no rule on the order or the number of these suffixes in a word. Make sure your grammar accepts everything from _hundo_ to _hundinetinegegeto_ (i.e. do not worry about the overgeneration).
1. The plural is marked by the _j_ suffix, which follows the _o_; e.g. _hundoj_
1. The accusative is marked by _n_; it is the last morpheme, e.g. in _hundojn_

Note: start simple, and try to make the test cells work one-by-one.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
def test_compile(grammar, fst_file):
    try:
        if os.path.exists(fst_file):
            os.unlink(fst_file)
        compile_lexc(grammar, fst_file)
        assert True
    except Exception as e:
        assert False, 'Got exception: {}'.format(e)
    
def words_valid(fst_file, words, up=True):
    for word in apply(fst_file, words, up):
        if word[1] == '+?':
            assert False, 'Word {} could not be parsed'.format(word[0])
    assert True
    
def words_invalid(fst_file, words, up=True):
    for word in apply(fst_file, words, up):
        if word[1] != '+?':
            assert False, 'Word {} should not have been parsed'.format(word[0])
    assert True

In [ ]:
# Test nominative
test_compile(esperanto_1, 'esperanto_1.fst')
words_valid('esperanto_1.fst', ['hundo', 'kato', 'birdo', 'elefanto'])
words_invalid('esperanto_1.fst', ['hund', 'kata', 'bird', 'elefantoo'])


In [ ]:
# Test plural
words_valid('esperanto_1.fst', ['hundoj', 'tigroj', 'leono', 'bestoj'])
words_invalid('esperanto_1.fst', ['hundj', 'tigr', 'leonojj', 'bestjo'])


In [ ]:
# Test accusative
words_valid('esperanto_1.fst', ['hundon', 'katojn', 'birdon'])
words_invalid('esperanto_1.fst', ['hundn', 'katonj', 'birdno'])


In [ ]:
# Test overgeneration
words_valid('esperanto_1.fst', ['elefantegojn', 'leonino', 'bestetino', 'hundinetinegegeto'])
words_invalid('esperanto_1.fst', ['elefantojegn', 'leonin', 'bestoetin', 'eghundinetinegeto'])


## 1.2 Male and female (5 points)

Esperanto animal names can take the _ge_ prefix. It means "male and female" (i.e. a pair of the animal in question). Add the prefix to your grammar. Examples are _getigregoj_ and _geelefantojn_. Also, make sure that:
- such nouns should always be plural (_j_), so no _gehundo_
- the _in_ suffix is forbidden, so _geleoninoj_ is invalid

The easiest way to do this would be with _flag diacritics_. This time, the `@U@` flag will probably not work; try to form an if-else structure with some of the others (e.g. `@P@` and `@D@`).

Note:
- make sure that the new grammar still accepts and rejects the same words as before if they don't start with _ge_
- completion of this exercise is **not** required to solve 1.3

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
test_compile(esperanto_2, 'esperanto_2.fst')
words_valid('esperanto_2.fst', ['gehundoj', 'gekatetegoj', 'gebirdetojn'])
words_valid('esperanto_1.fst', ['hundojn', 'tigro', 'leonino', 'elefantegetinegineto'])
words_invalid('esperanto_2.fst', ['geelefanto', 'getigrinoj'])


## 1.3 With tags (15 points)

It is time now to convert your Esperanto FSA to a full-fledged morphological analyzer FST. Incorporate the following tags into the grammar:

| Tag | Morpheme | Comment |
|-----|----------|---------|
| `MF+` | male-female | if implemented |
| `+Noun` | noun roots | all of them... |
| `+Fem` | feminine (_in_) | |
| `+Dim` | diminutive (_et_) | |
| `+Aug` | augmentative (_eg_) | |
| `+NSuff` | the _o_ suffix | |
| `+Pl` | plural (_j_) | |
| `+Sg` | singular | when not _j_ |
| `+Acc` | accusative (_n_) | |
| `+Nom` | nominative | when not _n_ |

The tags should **only** appear on the upper side; the morphemes (with the exception of roots) on the lower side. Some examples:

| Lexical (upper) | Surface (lower) |
|-----------------|-----------------|
| `hund+Noun+NSuff+Sg+Nom` | _hundo_ |
| `MF+best+Noun+Dim+NSuff+Pl+Acc` | _gebestetojn_ |

Note: you don't have to complete 1.2 to earn points for this exercise. Also, don't forget: "up" is left, "down" is right.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
def test_analysis(fst_file, input, output, up=True):
    res = apply(fst_file, [input], up)
    assert res[0][1] == output, '{} is not parsed as {} but {}'.format(input, output, res[0][1])

test_compile(esperanto_3, 'esperanto_3.fst')

test_analysis('esperanto_3.fst', 'hundo', 'hund+Noun+NSuff+Sg+Nom')
test_analysis('esperanto_3.fst', 'best+Noun+Dim+NSuff+Pl+Acc', 'bestetojn', False)
test_analysis('esperanto_3.fst', 'birdetegetegeteginoj', 'bird+Noun+Dim+Aug+Dim+Aug+Dim+Aug+Fem+NSuff+Pl+Nom')

words_valid('esperanto_3.fst', ['hund+Noun+NSuff+Pl+Acc', 'tigr+Noun+NSuff+Sg+Nom',
                                'leon+Noun+Fem+NSuff+Sg+Nom',
                                'elefant+Noun+Aug+Dim+Fem+Aug+Fem+Dim+NSuff+Sg+Nom'], False)
words_invalid('esperanto_3.fst', ['kat+Noun+NSuff+NSuff+Sg+Acc', 'bird+Noun+Dim+NSuff+Acc+Acc',
                                  'tigr+Noun+Pl+Nom', 'best+Acc', 'leon+NSuff+Dim+Sg+Nom'], False)


In [ ]:
test_analysis('esperanto_3.fst', 'gehundoj', 'MF+hund+Noun+NSuff+Pl+Nom')
test_analysis('esperanto_3.fst', 'gebirdetegojn', 'MF+bird+Noun+Dim+Aug+NSuff+Pl+Acc')
test_analysis('esperanto_3.fst', 'MF+best+Noun+Aug+NSuff+Pl+Acc', 'gebestegojn', False)


# Exercise 2: Toki Pona (47 points)

[Toki Pona](https://en.wikipedia.org/wiki/Toki_Pona) is a constructed language aimed at simplicity. It was designed to "shape the thought process of its users in a Zen-like fashion". In this exercise, your task will be to write a phrase structure grammar for Toki Pona.

Read the Wikipedia article linked above to familiarize yourself with the language.

## 2.1 Obtain the word list (10 points)

Toki Pona has (around) 120 root words, each describing a simple concept. These can be compounded (arranged into noun phrases) to express more complex ideas. Root words are not sorted into part-of-speech classes; instead, a root word can be a noun, verb or modifier based on where it occurs in the sentence.

Your first task is to get hold of the word list. The [canonical list](http://tokipona.net/tp/ClassicWordList.aspx) can be found on tokipona.net in HTML. Write a function that, given a URL, downloads the page and extracts the word list from it.

Your functions should also have a `remove_stop` argument. Read the [Syntax part of the Wikipedia page](https://en.wikipedia.org/wiki/Toki_Pona#Syntax) to see which words are used only to separate parts of the sentence from one another. This is almost everything in the syntax rules description that is written in **bold**, with the exception of _interjections_ and `mi` and `sina`, `lon` and `tawa`. If `remove_stop` is `True`, these words should be removed from the returned list (i.e. only content words are should be returned).

**Note:**: Do not use any third-party libraries for this; they might not be available to the grading environment. your code should work for all URL types; use the [urllib.request](https://docs.python.org/3/library/urllib.request.html) library.

In [ ]:
from collections import Counter

def get_tp_roots(url, remove_stop=False):
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
roots = get_tp_roots('http://tokipona.net/tp/ClassicWordList.aspx')

assert isinstance(roots, list) or isinstance(roots, set)
assert len(roots) == 125


In [ ]:
content_roots = get_tp_roots('http://tokipona.net/tp/ClassicWordList.aspx', True)
assert len(content_roots) == 114
assert 'mi' in content_roots
assert 'la' not in content_roots
assert 'lon' in content_roots
assert 'taso' not in content_roots


## 2.2 Grammaticality (12 points)

We will need functions to be able to test the grammar. Implement the following two functions:
- `is_grammatical` accepts a parser and a sentence, and check whether the sentence is grammatical
- `find_all_nt` accepts a `Tree` and a nonterminal (as a string). If returns a list of sentence fragments that are covered by the specified nonterminal. Study the [`Tree` API](http://www.nltk.org/_modules/nltk/tree.html#Tree) to find the function you need.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# For testing
test_grammar = nltk.CFG.fromstring("""
S -> NP VP
NP -> "n"
VP -> V NP | S
V -> "v"
""")
test_parser = nltk.ChartParser(test_grammar)

In [ ]:
assert is_grammatical(test_parser, 'nvn')
assert is_grammatical(test_parser, 'nnvn')
assert not is_grammatical(test_parser, 'vn')

# assert is_grammatical(tp, 'mama pi mi mute o sina lon sewi kon'.split())
# assert is_grammatical(tp, 'tenpo ali la sina jo e ma e wawa e pona'.split())
# assert not is_grammatical(tp, 'toki toki toki'.split())


In [ ]:
assert find_all_nt(test_parser.parse_one('nvn'), 'S') == ['n v n']
assert find_all_nt(test_parser.parse_one('nvn'), 'NP') == ['n'] * 2
assert find_all_nt(test_parser.parse_one('nnnvn'), 'S') == ['n n n v n', 'n n v n', 'n v n']
assert find_all_nt(test_parser.parse_one('nnnvn'), 'NP') == ['n'] * 4


## 2.3 A Toki Pona PSG grammar (25 points)

Following the [Syntax description](https://en.wikipedia.org/wiki/Toki_Pona#Syntax), write an PSG grammar for Toki Pona. Simply translate the ten rules to the format accepted by NLTK, with the following caveats:

- omit 8c and 8d
- the name of the nonterminals must be the same as in the grammar description, only in CamelCase (e.g. _Verbal_, _SubClause_)...
- ...with the exception of phrases and the sentence itself, which should have abbreviated names:
    - Sentence: _S_
    - Noun phrase: _NP_
    - Preposition phrase: _PP_
    - Verb phrase: _VP_
    - Simple noun phrase: _SNP_
- handle both optionality (_[]_) and repetition (_*_) in the grammar
- you might want to introduce nonterminals aside from the ones in the grammar description. You can do so, but always make sure their names end with an underscore (`_`), so that tree-comparison tests work
- don't forget to add the content roots to the grammar

You don't have to write the whole grammar at the same time. The task can be divided in many ways; one division is presented below.

### 2.3.1 Interjections

This grammar should implement 1a from the Wikipedia list.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# You have to create inj_grammar_str, above
tp_inj_grammar = nltk.CFG.fromstring(inj_grammar_str)

In [ ]:
tp_inj = nltk.ChartParser(tp_inj_grammar)

assert tp_inj
assert tp_inj_grammar.start().symbol() == 'S'

assert is_grammatical(tp_inj, 'a'.split())
assert is_grammatical(tp_inj, 'jaki'.split())
assert find_all_nt(tp_inj.parse_one('pakala'.split()), 'Interjection') == ['pakala']


### 2.3.2 Nouns

The next three parts concern the most important units in the grammar: noun phrases. We start with nouns. As we have previously seen, all content roots can act as nouns.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

tp_noun_grammar = nltk.CFG.fromstring(noun_grammar_str)

In [ ]:
tp_noun = nltk.ChartParser(tp_noun_grammar)

assert tp_noun_grammar.start().symbol() == 'Noun'

assert is_grammatical(tp_noun, 'akesi'.split())
assert is_grammatical(tp_noun, 'jaki'.split())
assert tp_noun.parse_one('monsi'.split()).pos()[0][1] == 'Noun'


### 2.3.3 Simple noun phrases

Now that we know what our nouns are, we can put them into simple noun phrases. This concerns parts 6a and b from the Wikipedia page.

Forget the `tp_noun_grammar` object from above, but incorporate `noun_grammar_str` into your new `snp_grammar_str`, below. Also, you must have to handle _modifiers_ as well $-$ again, all content roots can behave as modifiers.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

tp_snp_grammar = nltk.CFG.fromstring(snp_grammar_str)

In [ ]:
import random

tp_snp = nltk.ChartParser(tp_snp_grammar)

assert tp_snp_grammar.start().symbol() == 'SNP'

# Nouns are SNPs
assert is_grammatical(tp_snp, 'akesi'.split())
assert is_grammatical(tp_snp, 'jaki'.split())
assert tp_snp.parse_one('monsi'.split()).pos()[0][1] == 'Noun'

# Noun + modifier
assert is_grammatical(tp_snp, 'jan utala'.split())
assert is_grammatical(tp_snp, 'kasi kule'.split())
little_dog = tp_snp.parse_one('soweli lili'.split())
assert [pos for word, pos in little_dog.pos()] == ['Noun', 'Modifier']

# Noun + modifiers
assert is_grammatical(tp_snp, random.sample(content_roots, k=5))
little_dog_eating = tp_snp.parse_one('soweli lili moku'.split())
assert find_all_nt(little_dog_eating, 'Noun') == ['soweli']
assert find_all_nt(little_dog_eating, 'Modifier') == ['lili', 'moku']

# pi
assert is_grammatical(tp_snp, 'mama pi mi mute'.split())
tree = tp_snp.parse_one('poki kule pi soweli lili moku'.split())
assert find_all_nt(tree, 'Noun') == ['poki', 'soweli']
assert find_all_nt(tree, 'Modifier') == ['kule', 'lili', 'moku']


### 2.3.4 Noun phrases

This is just 6c on top of the SNP.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

tp_np_grammar = nltk.CFG.fromstring(np_grammar_str)

In [ ]:
tp_np = nltk.ChartParser(tp_np_grammar)

assert tp_np_grammar.start().symbol() == 'NP'

# SNPs are NPs
assert is_grammatical(tp_np, 'akesi'.split())
assert tp_np.parse_one('monsi'.split()).pos()[0][1] == 'Noun'
little_dog = tp_np.parse_one('soweli lili'.split())
assert [pos for word, pos in little_dog.pos()] == ['Noun', 'Modifier']
assert is_grammatical(tp_np, random.sample(content_roots, k=5))
tree = tp_np.parse_one('poki kule pi soweli lili moku'.split())
assert find_all_nt(tree, 'Noun') == ['poki', 'soweli']
assert find_all_nt(tree, 'Modifier') == ['kule', 'lili', 'moku']

# Conjunction
assert is_grammatical(tp_np, 'akesi anu poki kule pi soweli lili moku'.split())
tree = tp_np.parse_one('soweli lili moku pi jan utala en poki kule pi soweli lili'.split())
assert find_all_nt(tree, 'Noun') == ['soweli', 'jan', 'poki', 'soweli']
assert find_all_nt(tree, 'SNP') == ['soweli lili moku pi jan utala', 'soweli lili moku',
                                    'poki kule pi soweli lili', 'poki kule']


### 2.3.5 Subjects, objects, etc.

With the NP at hand, you can now build some of the other structures in the grammar:
- subjects (4)
- direct objects (10)
- vocatives (3)
- prepositional phreses (7)

We cannot test them in the same grammar, so we need to create four grammars this time.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

tp_subj_grammar = nltk.CFG.fromstring(subj_grammar_str)
tp_obj_grammar = nltk.CFG.fromstring(obj_grammar_str)
tp_voc_grammar = nltk.CFG.fromstring(voc_grammar_str)
tp_pp_grammar = nltk.CFG.fromstring(pp_grammar_str)

In [ ]:
tp_subj = nltk.ChartParser(tp_subj_grammar)
tp_obj = nltk.ChartParser(tp_obj_grammar)
tp_voc = nltk.ChartParser(tp_voc_grammar)
tp_pp = nltk.ChartParser(tp_pp_grammar)

assert tp_subj_grammar.start().symbol() == 'Subject'
assert tp_obj_grammar.start().symbol() == 'DirectObject'

# Subjects
assert is_grammatical(tp_subj, 'mi'.split())
assert tp_subj.parse_one('monsi li'.split()).pos()[0][1] == 'Noun'
tree = tp_subj.parse_one('poki kule pi soweli lili moku li'.split())
assert find_all_nt(tree, 'NP') == ['poki kule pi soweli lili moku']
assert find_all_nt(tree, 'Subject') == ['poki kule pi soweli lili moku li']

# Direct objects
assert is_grammatical(tp_obj, 'e poki kule pi soweli lili moku'.split())
assert not is_grammatical(tp_obj, 'e akesi anu poki kule pi soweli lili moku'.split())
tree = tp_obj.parse_one('e poki kule pi soweli lili moku pi jan utala'.split())
assert find_all_nt(tree, 'SNP') == ['poki kule pi soweli lili moku pi jan utala',
                                    'poki kule pi soweli lili moku', 'poki kule']
assert tree.label() == 'DirectObject'

# Vocatives
tree = tp_voc.parse_one('akesi anu poki kule pi soweli lili moku o'.split())
assert find_all_nt(tree, 'Vocative') == ['akesi anu poki kule pi soweli lili moku o']

# Prepositional phrases
assert tp_pp.parse_one('kepeken akesi en soweli lili'.split()).pos()[0][1] == 'Preposition'


### 2.3.6 Verb phrases

Verb phrases (8, 9) are a bit more hairy, but nothing earth-shattering. As mentioned before, skip 8c and d. Don't forget to append the `DirectObject` grammar string you created previously.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

tp_vp_grammar = nltk.CFG.fromstring(vp_grammar_str)

In [ ]:
assert tp_vp_grammar.start().symbol() == 'VP'

tp_vp = nltk.ChartParser(tp_vp_grammar)

assert tp_vp.parse_one('lape'.split()).pos()[0][1] == 'Verb'
assert tp_vp.parse_one('lape pona'.split()).pos()[1][1] == 'Modifier'
tree = tp_vp.parse_one('moku e moku seli'.split())
assert find_all_nt(tree, 'DirectObject') == ['e moku seli']
assert find_all_nt(tree, 'SNP') == ['moku seli']


### 2.3.7 Prepositions

Prepositions incorporate the verb and preposition phrases (5). Don't forget to name the conjunction nonterminal occuring here differently from the one you used in the NP grammar; otherwise, the two rules will conflict.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

tp_pred_grammar = nltk.CFG.fromstring(pred_grammar_str)

In [ ]:
assert tp_pred_grammar.start().symbol() == 'Predicate'

tp_pred = nltk.ChartParser(tp_pred_grammar)

tree = tp_pred.parse_one('moku e moku seli'.split())
assert find_all_nt(tree, 'Predicate') == find_all_nt(tree, 'VP') == ['moku e moku seli']

tree = tp_pred.parse_one('soweli lili pi jan utala'.split())
assert find_all_nt(tree, 'Predicate') == ['soweli lili pi jan utala']
assert len(find_all_nt(tree, 'SNP')) == 2

assert not is_grammatical(tp_pred, 'soweli lili en jan utala'.split())
assert is_grammatical(tp_pred, 'soweli lili anu jan utala'.split())
tree = tp_pred.parse_one('tomo suli li awen jaki'.split())
assert find_all_nt(tree, 'Predicate') == ['tomo suli li awen jaki', 'tomo suli', 'awen jaki']


### 2.3.8 Grand slam

Incorporate the rest into your grammar: sentences and sub-clauses (1 and 2). Don't forget to append the grammar strings you have created thus far (you might not need all, e.g. `np` is part of `pred`, etc.).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

tp_grammar = nltk.CFG.fromstring(tp_grammar_str)

In [ ]:
assert tp_grammar.start().symbol() == 'S'

tp = nltk.ChartParser(tp_grammar)

trees = list(tp.parse('mama pi mi mute o sina lon sewi kon'.split()))
assert len(trees) == 11
assert ['mama', 'mi', 'sewi'] in [find_all_nt(t, 'Noun') for t in trees]
assert ['lon'] in [find_all_nt(t, 'Verb') for t in trees]
assert ['sina'] in [find_all_nt(t, 'Verb') for t in trees]

trees = list(tp.parse('tenpo ali la sina jo e ma e wawa e pona'.split()))
assert len(trees) == 1
assert find_all_nt(trees[0], 'DirectObject') == ['e ma', 'e wawa', 'e pona']

assert is_grammatical(tp, 'toki'.split())
assert not is_grammatical(tp, 'toki toki'.split())
assert is_grammatical(tp, 'jan meli li toki e toki pona'.split())
assert is_grammatical(tp, 'soweli seme mu li utala ala'.split())
assert is_grammatical(tp, 'mi mute li lape e lipu'.split())
assert is_grammatical(tp, 'waso li olin e pan li tawa wawa lon kon'.split())
assert is_grammatical(tp, 'jan lili laso li lon'.split())
assert is_grammatical(tp, 'jan li tawa kepeken kepeken'.split())
assert is_grammatical(tp, 'jan li tawa kepeken noka'.split())
assert is_grammatical(tp, 'kute li lon'.split())
assert not is_grammatical(tp, 'kute li'.split())

### 2.3.9 Grammar properties

Compute the three numbers printed below. Lexical productions are those that have a single terminal (`str`) on the right hand side (i.e. the `A` $\rightarrow$ `a` format).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print('Total number of productions:', num_all_productions)
print('Number of lexical productions:', num_lex_productions)
print('Number of nonterminals', num_nt)


### 2.3.10 Normalize your grammar

If you created your grammar by concatenating separate grammar strings, you probably ended up with a lot of productions. The reason for this is that many rules were present in more than one grammar string. Normalize your grammar by making sure each line occurs only once in `tp_grammar_str`.

Compute the three statistics for the normalized grammar and see how the number of productions goes down.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print('Total number of productions (norm):', num_all_productions_norm)
print('Number of lexical productions (norm):', num_lex_productions_norm)
print('Number of nonterminals (norm)', num_nt_norm)

assert num_nt == num_nt_norm


# Exercise 3: Grammar induction (20 points)

In this exercise, you will parse a treebank, and induce a PCFG grammar from it. Finally, you will use your grammar to compute the probability of the trees in the training corpus.

You need a probabilistic version of the CKY algorithm to parse sentences with a PCFG. PCKY is not part of this homework; however, if you are interested, you can try to implement it based on the CKY code you created during the lab exercises.

## 3.1 Parse a treebank (10 points)

Parse the treebank file `en_lines-ud-train.s` in the notebook's directory. Write a **generator** function that reads the file and yields `nltk.tree.Tree` objects. In particular,
- do not read the whole file into memory
- the `Tree.fromstring()` function converts an s-expression into a tree

Open the file in an editor to see the formatting.

Note that the file was created by parsing the [LinES dependency corpus](https://github.com/UniversalDependencies/UD_English-LinES/tree/master) with [Stanford CoreNLP](https://stanfordnlp.github.io/CoreNLP/), so it is not a gold standard by any means, but it will suffice for now.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
trees = parse_treebank('en_lines-ud-train.s')

assert isinstance(trees, types.GeneratorType)
assert sum(1 for _ in parse_treebank('en_lines-ud-train.s')) == 2613
assert isinstance(next(parse_treebank('en_lines-ud-train.s')), Tree)


## 3.2 Filter trees (5 points)

In order to avoid problems further down the line, we shall only handle a subset of the trees in the treebank. We call a tree _valid_, if
- its root is `'S'`
- the root has at least two children.

Write a function that returns `True` for "valid" trees and `False` for invalid ones. Filter the your generator with it.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
filtered_trees = filter(is_tree_valid, trees)

assert sum(map(is_tree_valid, filtered_trees)) == 2311

## 3.3 Induce the PCFG grammar (5 points)

Now that you have the trees, it is time to induce (train) a PCFG grammar for it! Luckily, `nltk` has a functions for just that: [`nltk.grammar.induce_pcfg`](http://www.nltk.org/api/nltk.html#nltk.grammar.induce_pcfg). Use it to acquire your PCFG grammar. You can find hints at how to use it in the [grammar module](http://www.nltk.org/_modules/nltk/grammar.html).

Note: since we want to parse sentences with the PCKY algorithm, we need our grammar to be in CNF. Unfortunately, `nltk` cannot convert a grammar to CNF, so you have to ensure that the trees are in CNF before feeding them to the PCFG induction function. That way, we can be sure that our grammar will be also. There are two functions that ensure a tree is in CNF:
- [`collapse_unary`](http://www.nltk.org/api/nltk.html#nltk.tree.Tree.collapse_unary). Make sure you call it with `collapsePOS=True`!
- [`chomsky_normal_form`](http://www.nltk.org/api/nltk.html#nltk.tree.Tree.chomsky_normal_form). Do not use any smoothing.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
grammar = train_grammar(filter(is_tree_valid, parse_treebank('en_lines-ud-train.s')))

assert len(grammar.productions()) == 15000


## 3.4 *Bonus exercise: training corpus probability (10 points)

Write a function that, given a PCFG and a parse tree, computes the probability of the latter. Be sure to use and return [log probability](https://en.wikipedia.org/wiki/Log_probability), otherwise the probabilities may become too small for `float` to manage. Probabilistic production objects support both the `prob` and the `logprob` methods.

The trees shown to the function might not be in CNF, so make sure you convert them before computing the probability.

Note: store production probabilities in a (global) dictionary to speed up your function. The second test cell will run very long if you use just regular `nltk` functions -- certainly too long for the autograder!

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
trees = list(filter(is_tree_valid, parse_treebank('en_lines-ud-train.s')))

assert numpy.allclose(tree_logprob(grammar, trees[0].copy(True)), -214.22993036407965)
assert numpy.allclose(tree_logprob(grammar, trees[1000].copy(True)), -60.930244885517915)
assert numpy.allclose(tree_logprob(grammar, trees[2000].copy(True)), -211.13780770649575)


In [ ]:
tree_logprobs = [tree_logprob(grammar, tree.copy(True)) for tree in trees]

min_logprob = min(tree_logprobs)
max_logprob = max(tree_logprobs)
sum_logprob = sum(tree_logprobs)

print('Min logprob:', min_logprob)
print('Max logprob:', max_logprob)
print('Corpus probability:', sum_logprob)

assert numpy.allclose(min_logprob, -1050.314868185599)
